In [1]:
# Imports 
import os
import sys
from io import StringIO
sys.path.append("..")
os.environ["pyna_language"] = 'julia'
import pdb
import numpy as np
import openmdao.api as om
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
%matplotlib inline
from pyNA.pyna import pyna

from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Load 
epnl_sideline = dict()
epnl_lateral = dict()
epnl_flyover = dict()

lat_att = False

for ac_name in ['stca', 'a10']:
    
    epnl_sideline[ac_name] = dict()
    epnl_lateral[ac_name] = dict()
    epnl_flyover[ac_name] = dict()
    
    for case in ['V2', 'Vmax']:
    
        if ac_name == 'stca' and case == 'V2':
            start = 20
        elif ac_name == 'stca' and case == 'Vmax':
            start = 35
        elif ac_name == 'a10' and case == 'V2':
            start = 60
        elif ac_name == 'a10' and case == 'Vmax':
            start = 70

        epnl_flyover[ac_name][case] = np.load('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps/epnl_flyover.npy') 
        epnl_sideline[ac_name][case] = np.load('../cases/'+ac_name+'/output/STCB/'+case+'/sweeps/epnl_sideline.npy') 
        epnl_lateral[ac_name][case] = np.max(epnl_sideline[ac_name][case][:,:,start:], axis=2)

In [3]:
# Inputs 
z_cb_lst = np.hstack((np.linspace(25, 250, 10), np.linspace(300, 750, 10)))

TS_cb_lst = dict()
TS_cb_lst['stca']= dict()
TS_cb_lst['stca']['V2'] = np.linspace(0.75, 1.0, 11)
TS_cb_lst['stca']['Vmax'] = np.linspace(0.4, 1.0, 25)
TS_cb_lst['a10'] = dict()
TS_cb_lst['a10']['V2'] = np.linspace(0.775, 1.0, 10)
TS_cb_lst['a10']['Vmax'] = np.linspace(0.675, 1.0, 14)

Vmax_lst = dict()
Vmax_lst['stca'] = dict()
Vmax_lst['stca']['V2'] = 197*0.51444444
Vmax_lst['stca']['Vmax'] = 250*0.51444444
Vmax_lst['a10'] = dict()
Vmax_lst['a10']['V2'] = 234.4*0.51444444
Vmax_lst['a10']['Vmax'] = 250*0.51444444

py = pyna()
epnl_constraint = dict()
epnl_constraint['stca'] = dict() 
epnl_constraint['stca']['V2']   = 93.88114249-0.5
epnl_constraint['stca']['Vmax'] = 91.74406241-0.5
epnl_constraint['a10'] = dict()
epnl_constraint['a10']['V2']   = 96.07793493-0.5
epnl_constraint['a10']['Vmax'] = 94.27878407-0.5

# PTCB trajectories

In [4]:
# Load verification trajectory 
nasa_std = pd.read_csv('../cases/nasa_stca_standard/trajectory/Trajectory_to.csv')

if True:
    
    for ac_name in ['stca']:
        for ii, case in enumerate(['V2']):

            py = pyna()
            py.initialize()
            
            if True:

                py.ac_name = ac_name
                py.case_name = ac_name
                py.engine_deck_name = 'engine_deck_'+ac_name+'.csv'
                py.TS_to = 1.
                py.TS_vnrs = TS_cb_lst[ac_name][case][0]
                py.TS_cb = TS_cb_lst[ac_name][case][0]
                py.save_results = True
                py.levels_int_metric = 'ptcb'
                py.output_directory_name = 'PTCB/'+case
                py.output_file_name = ac_name+'_ptcb_'+case+'.sql'
                
                py.noise_constraint_lateral = epnl_constraint[ac_name][case]
                py.ptcb = True
                py.pkrot = True
                py.phld = False
                py.v_max = Vmax_lst[ac_name][case]
                
                py.all_sources = False
                py.fan_inlet_source = True
                py.fan_discharge_source = True
                py.core_source = True
                py.jet_mixing_source = True
                py.jet_shock_source = False
                py.airframe_source = False
                
                # Simplifications
                py.n_frequency_subbands = 1
                py.max_iter = 300

                if ac_name == 'stca' and case == 'V2':
                    x_observer_array = np.linspace(1000, 6250, 16)
                elif ac_name == 'stca' and case == 'Vmax':
                    x_observer_array = np.linspace(1000, 6250, 16)
                elif ac_name == 'a10' and case == 'V2':
                    x_observer_array = np.linspace(2300, 6500, 13)
                elif ac_name == 'a10' and case == 'Vmax':
                    x_observer_array = np.linspace(2300, 6500, 13)

            if True:                  
                py.optimize_trajectory_noise(x_lateral_observer=x_observer_array, initialization_path_name=None)

            if True:
                results = py.load_results(file_name=ac_name+'_ptcb_'+case+'.sql')

                path_save_name = 'trajectory_' + ac_name + '_ptcb_' + case + '.csv'
                engine_save_name = 'engine_' + ac_name + '_ptcb_' + case + '.csv' 
                py.save_time_series(results, py.airframe, path_save_name, engine_save_name)

            if True:
                n_sl = 131  

                py.engine_timeseries_name = 'engine_'+ac_name+'_ptcb_'+case+'.csv'
                py.trajectory_file_name = 'trajectory_'+ac_name+'_ptcb_'+case+'.csv'
                py.output_file_name = ac_name+'_ptcb_'+case+'_noisetimeseries.sql'
                py.observer_lst = ['contour',]
                py.levels_int_metric = 'epnl'

                py.all_sources = False
                py.fan_inlet_source = True
                py.fan_discharge_source = True
                py.core_source = True
                py.jet_mixing_source = True
                py.jet_shock_source = False
                py.airframe_source = True

                x_observer_lst = np.zeros((n_sl+1, 3))
                x_observer_lst[:-1, 0] = np.linspace(0, 6500, n_sl)
                x_observer_lst[:-1, 1] = 450*np.ones(n_sl)
                x_observer_lst[:, 2] = 4*0.3048*np.ones(n_sl+1)
                x_observer_lst[-1,0] = 6500.

                py.x_observer_array = x_observer_lst
                py.compute_noise_timeseries()

                epnl_sideline = py.noise_timeseries.get_val('noise.epnl')[:-1]
                epnl_lateral = np.max(py.noise_timeseries.get_val('noise.epnl')[:-1])
                epnl_flyover = py.noise_timeseries.get_val('noise.epnl')[-1]

            if True:
                np.save('../cases/'+ac_name+'/output/PTCB/'+case+'/epnl_lateral.npy', epnl_lateral)
                np.save('../cases/'+ac_name+'/output/PTCB/'+case+'/epnl_flyover.npy', epnl_flyover)
                np.save('../cases/'+ac_name+'/output/PTCB/'+case+'/epnl_sideline.npy', epnl_sideline)

/Users/laurensvoet/Library/Python/3.7/lib/python/site-packages/openmdao/utils/options_dictionary.py:332: OMDeprecationWarning:'gradient method' is not a valid python name and will become an invalid option name in a future release. You can prevent this warning (and future exceptions) by declaring this option using a valid python name.


--- Linkage Report [phases] ---
    --- groundroll - rotation ---
        time               [final] ==  time           [initial]
        states:x           [final] ==  states:x       [initial]
        states:v           [final] ==  states:v       [initial]
        parameters:alpha   [final*] ==  states:alpha   [initial]
    --- rotation - liftoff ---
        time               [final] ==  time             [initial]
        states:x           [final] ==  states:x         [initial]
        parameters:z       [final*] ==  states:z         [initial]
        states:v           [final] ==  states:v         [initial]
        states:alpha       [final] ==  controls:alpha   [initial]
        parameters:gamma   [final*] ==  states:gamma     [initial]
    --- liftoff - vnrs ---
        time             [final] ==  time             [initial]
        states:x         [final] ==  states:x         [initial]
        states:v         [final] ==  states:v         [initial]
        controls:alpha   [fin

/Users/laurensvoet/Library/Python/3.7/lib/python/site-packages/openmdao/recorders/sqlite_recorder.py:224: UserWarning:The existing case recorder file, /Users/laurensvoet/Documents/Research/pyNA/pyNA/cases/stca/output/PTCB/V2/stca_ptcb_V2.sql, is being overwritten.
/Users/laurensvoet/Library/Python/3.7/lib/python/site-packages/openmdao/recorders/sqlite_recorder.py:224: UserWarning:The existing case recorder file, dymos_solution.db, is being overwritten.


Model viewer data has already has already been recorded for Driver.
Model viewer data has already has already been recorded for Driver.
Full total jacobian was computed 3 times, taking 6.651734 seconds.
Total jacobian shape: (252, 176) 


Jacobian shape: (252, 176)  ( 3.84% nonzero)
FWD solves: 11   REV solves: 0
Total colors vs. total size: 11 vs 176  (93.8% improvement)

Sparsity computed using tolerance: 1e-12
Time to compute sparsity: 6.651734 sec.
Time to compute coloring: 0.252391 sec.


Optimization Problem -- Optimization using pyOpt_sparse
    Objective Function: _objfunc

    Solution: 
--------------------------------------------------------------------------------
    Total Time:                   12.8111
       User Objective Time :       1.7020
       User Sensitivity Time :     9.9967
       Interface Time :            1.0081
       Opt Solver Time:            0.1043
    Calls to Objective Function :      22
    Calls to Sens Function :           22


   Objectives
     

--- Linkage Report [phases] ---
    --- groundroll - rotation ---
        time               [final] ==  time           [initial]
        states:x           [final] ==  states:x       [initial]
        states:v           [final] ==  states:v       [initial]
        parameters:alpha   [final*] ==  states:alpha   [initial]
    --- rotation - liftoff ---
        time               [final] ==  time             [initial]
        states:x           [final] ==  states:x         [initial]
        parameters:z       [final*] ==  states:z         [initial]
        states:v           [final] ==  states:v         [initial]
        states:alpha       [final] ==  controls:alpha   [initial]
        parameters:gamma   [final*] ==  states:gamma     [initial]
    --- liftoff - vnrs ---
        time             [final] ==  time             [initial]
        states:x         [final] ==  states:x         [initial]
        states:v         [final] ==  states:v         [initial]
        controls:alpha   [fin

RuntimeError: 'noise' <class JuliaExplicitComp>: Error calling compute(), <PyCall.jlwrap (in a Julia function called from Python)
JULIA: UndefVarError: kkk not defined
Stacktrace:
 [1] get_noise!(::Array{Float64,1}, ::Array{Float64,1}, ::Dict{Any,Any}, ::PynaInterpolations, ::PyObject, ::PyObject, ::Dict{Any,Any}, ::Int64, ::Int64) at /Users/laurensvoet/Documents/Research/pyNA/pyNA/src/noise_src_jl/get_noise.jl:228
 [2] compute!(::NoiseModel{Dict{Any,Any},PynaInterpolations,PyObject,Dict{Any,Any},PyObject,Int64,Int64,String}, ::PyDict{String,PyArray,true}, ::PyDict{String,PyArray,true}) at /Users/laurensvoet/Documents/Research/pyNA/pyNA/src/noise_src_jl/noise_model.jl:409
 [3] compute!(::BigInt, ::PyDict{String,PyArray,true}, ::PyDict{String,PyArray,true}) at /Users/laurensvoet/Documents/Research/OpenMDAO.jl/src/OpenMDAO.jl:84
 [4] (::PyCall.FuncWrapper{Tuple{Integer,PyDict{String,PyArray,isdict} where isdict,PyDict{String,PyArray,isdict} where isdict},typeof(compute!)})(::BigInt, ::Va